In [2]:
import torch

from vq_vae_text.datasets import LMDBDataset, ByteLevelTextDataset

In [53]:
s1 = torch.load('../results/textcnn-ref-2/checkpoints/model_3.pth').cpu()
s2 = torch.load('../results/2020-04-14_22-18-18-textcnnV2/checkpoints/model_0.pth').cpu()

/home/kris/miniconda3/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'vq_vae_text.models.textcnnv2.Decoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [54]:
text_data = ByteLevelTextDataset('/home/kris/data/text/sent-grams/splits/uniform/bigrams/small/test.txt', 256)

batch = torch.stack([text_data[i] for i in range(16)])

Using cached file .cache/da19f7d126ad83037dfa74cbb07e335086df0b2d8f75d7ffa6dc1887a76eaa1a.p


In [55]:
z, diff, lat1 = s1.encode(batch)
z, diff, lat2 = s2.encode(lat1)

lat2_dec = s2.decode_codes(lat2).argmax(-1)
lat1_dec = s1.decode_codes(lat2_dec).argmax(-1)

RuntimeError: Given transposed=1, weight of size [256, 256, 4], expected input[16, 4, 16] to have 256 channels, but got 4 channels instead

In [49]:
for i in range(100):
    lat2[0] = i
    
    lat2_dec = s2.decode_codes(lat2).argmax(-1)
    lat1_dec = s1.decode_codes(lat2_dec).argmax(-1)

    print(ByteLevelTextDataset.seq_to_text(lat1_dec)[0])

Laut eine muhsendung eine muhsendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine gessendung eine muhsendung eine Proan
Trosisch rusdrigkrastand rustringen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And en. Si
Trosisch rusdrigkrastand rustringen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And en. Si
Trosisch rusdrigkrastand rustringen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And end Jungen. And

In [35]:
s2.decode_codes(lat2).shape

torch.Size([16, 64, 4096])

In [ ]:
dataset = LMDBDataset('../results/textcnn-ref-2/train-latents')

ae = torch.load('../results/textcnn-ref-2/checkpoints/model_3.pth')
ae = ae.to('cpu')
ae.eval();

In [4]:
latents = torch.stack([dataset[i] for i in range(10)])

In [33]:
def latent_to_text(latent):
    recon = ae.decode_codes(latent).argmax(-1)
    return ByteLevelTextDataset.seq_to_text(recon)


In [6]:
latent_to_text(latents)

['Sa bin ich als Versehen inmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.\x01',
 'Ausschliesslich an Entwickler hat BRA einen als New Alpaa bezeichneten Prolotop ausgeliefert. Erste Bests mit weiteren Geraten, die London und Netadt heissen sollen, laufen angeblich bereits.\x01',
 'Sie liegt auf der Linie dessen, was die Bischafe immer gesagt hatten. Es gebe ein eigenstandiges Arbeitstecht fur die Kirche, und die Mitarbeiter wussten, dass sie sich daran zu halten haben.\x01',
 'Die Zahlen seien zwar im Rahmen der Erwartungen ausgefallen. Allerdings gebe es weder bei den Ergebnissen noch beim Ausblick irgendwelche aussergewohnlichen Uberraschungen, sagte ein Handler.\x01',
 'Die Zahlen seien zwar im Rahmen der Erwartungen ausgefallen. Allerdings gebe es weder bei den Ergebnissen noch beim Ausblick irgendwelche aussergewohnlichen Uberraschungen, sagte ein Handler.\x01',


In [7]:
print(latent_to_text(latents.view(1, -1))[0])

Sa bin ich als Versehen inmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.ostnlseriseriseriseriseriseriseriseriseriseriseriseryserystristfumschliesslich an Entwickler hat BRA einen als New Alpaa bezeichneten Prolotop ausgeliefert. Erste Bests mit weiteren Geraten, die London und Netadt heissen sollen, laufen angeblich bereits.ystniseriseriseriseriseriseriseriseriseriseriseriseryserystrrserie liegt auf der Linie dessen, was die Bischafe immer gesagt hatten. Es gebe ein eigenstandiges Arbeitstecht fur die Kirche, und die Mitarbeiter wussten, dass sie sich daran zu halten haben.ostilseriseriseriseriseriseriseriseriseriseriseriseryserystrrstrie Zahlen seien zwar im Rahmen der Erwartungen ausgefallen. Allerdings gebe es weder bei den Ergebnissen noch beim Ausblick irgendwelche aussergewohnlichen Uberraschungen, sagte ein Handler.ystnlseriseriseriseriseriseriseriserise

In [8]:
mix_latent = torch.cat([latents[0][:32], latents[1][32:]]).unsqueeze(0)
print(latent_to_text(mix_latent)[0])

Sa bin ich als Versehen inmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld re London und Netadt heissen sollen, laufen angeblich bereits.


In [9]:
step = 10
for i in range(32):
    latent = latents[0]
    latent[0] = i
    latent[5] = i % step
    
    if i % step == 0:
        print('\n')
    
    print(i, '\t' + latent_to_text(latent.view(1, -1))[0])



0 	22 Von ich als Verses. Fenditten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
1 	Ste in Wch als Vernahte inmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
2 	Jungon ich als Vernphungsnmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
3 	, plan ich als Verses flamkitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
4 	Otesch ich als Versahmesinmitten einer Hangriffe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
5 	Dw  an ich als 

In [12]:
for i in range(16):
    for j in range(16):
        latent = latents[0]
        latent[0] = i
        latent[5] = j
        
        print(j, '\t' + latent_to_text(latent.view(1, -1))[0])
    print('\n')

0 	22 Von ich aldsnassas. Henditten einer HDwsnrupe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
1 	22 Von ich alfsvornuhte inmitten einer HDwsnrupe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
2 	22 Von ich alfszarsphungsnmitten einer HDwsnrupe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
3 	22 Von ich aldsnissah flamkitten einer HDwsnrupe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
4 	22 Von ich altsvabnthmesinmitten einer HDwsnrupe mit dem Zug am Stadion angekommen. Die Einheimischen haben uns mit Geuchtgegeld beschossen und mit Steinen die Scheiben im Zug eingeschmissen.
5 	22 Von ich alfsno